In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.9 MB/s eta 0:00:00


In [ ]:
import os, csv, json
from ultralytics import YOLO
import cv2
from datetime import datetime, timezone

# Assuming AlertManager is defined in the environment or a preceding cell
# from alert_manager import AlertManager # This line will be removed or commented out

# Define a placeholder AlertManager class if it's not defined elsewhere
# In a real scenario, you would replace this with the actual implementation of AlertManager
class AlertManager:
    def __init__(self):
        print("AlertManager initialized.")

    def handle_violation(self, violation_data):
        print(f"Alert triggered for violation: {violation_data}")


# Initialize model + alert system
model = YOLO("/content/drive/MyDrive/Project/Dataset/css-data/cross_validation/results_fold3/weights/best.pt")
alert_mgr = AlertManager()

# Paths
video_dir = "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/visual_demo/outputs/alerts"
meta_dir = "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system/outputs/metaData"
os.makedirs(meta_dir, exist_ok=True)

meta_csv = os.path.join(meta_dir, "detection_metadata.csv")
meta_json = os.path.join(meta_dir, "detection_metadata.json")

# Initialize CSV if not exists
if not os.path.exists(meta_csv):
    with open(meta_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["timestamp", "video", "frame", "worker_id", "violation", "confidence"])

all_metadata = []

# Loop through all videos
for video_file in os.listdir(video_dir):
    if not video_file.endswith((".mp4", ".avi", ".mov")):
        continue

    video_path = os.path.join(video_dir, video_file)
    print(f"Processing: {video_file}")
    cap = cv2.VideoCapture(video_path)
    frame_id = 0
    names = model.names

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_id += 1
        results = model(frame, verbose=False)

        for box in results[0].boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            label = names[cls_id]
            worker_id = f"{video_file}_W{frame_id:05d}_{cls_id}"

            if "NO-" in label:  # violation detected
                timestamp = datetime.utcnow().replace(tzinfo=timezone.utc).isoformat()

                #Log metadata
                row = [timestamp, video_file, frame_id, worker_id, label, round(conf, 3)]
                with open(meta_csv, "a", newline="") as f:
                    csv.writer(f).writerow(row)

                #Append to JSON data
                all_metadata.append({
                    "timestamp": timestamp,
                    "video": video_file,
                    "frame": frame_id,
                    "worker_id": worker_id,
                    "violation": label,
                    "confidence": round(conf, 3)
                })

                #Alert trigger
                alert_mgr.handle_violation({
                    "video": video_file,
                    "frame": frame_id,
                    "violations": [label],
                    "timestamp": timestamp
                })

    cap.release()

#Save metadata JSON
with open(meta_json, "w") as f:
    json.dump(all_metadata, f, indent=4)

print(f"\nMetadata collection complete!\nCSV: {meta_csv}\nJSON: {meta_json}")

In [10]:
import os
import cv2
import time
import queue
import threading
import torch
from datetime import datetime, timezone
from ultralytics import YOLO

VIDEO_PATH = "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/testVideos/1100493329-preview.mp4"
MODEL_PATH = "/content/drive/MyDrive/Project/Dataset/css-data/cross_validation/results_fold3/weights/best.pt"
OUTPUT_META = "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system/outputs/metaData"
os.makedirs(OUTPUT_META, exist_ok=True)

FRAME_SKIP = 3             # process every 3rd frame
FRAME_WIDTH = 1280         # resize to 720p width (adjust as needed)
GPU = torch.cuda.is_available()

# ======================
# ⚙️ INITIALIZATION
# ======================
model = YOLO(MODEL_PATH)
alert_mgr = AlertManager()
names = model.names

cap = cv2.VideoCapture(VIDEO_PATH)
frame_queue = queue.Queue(maxsize=10)
stop_flag = threading.Event()

# Declare variables for tracking total time and processed frames in the global scope
start_time = time.time() # Global start time for detection
processed_frames = 0 # Global counter for processed frames

def video_reader():
    """Continuously reads frames from the video and stores them in a queue."""
    frame_id = 0
    while not stop_flag.is_set():
        ret, frame = cap.read()
        if not ret:
            stop_flag.set()
            break
        frame_id += 1

        # Frame skipping
        if frame_id % FRAME_SKIP != 0:
            continue

        # Resize to improve speed
        h, w = frame.shape[:2]
        new_h = int(h * FRAME_WIDTH / w)
        frame = cv2.resize(frame, (FRAME_WIDTH, new_h))

        frame_queue.put((frame_id, frame))
    cap.release()

def detection_worker():
    """Processes frames from the queue, performs inference, and triggers alerts."""
    global processed_frames # Declare global to modify the global variable

    # Use a local counter and start time for internal FPS reporting within the thread
    start_time_local_for_fps_reporting = time.time()
    processed_frames_local_for_fps_reporting = 0

    while not stop_flag.is_set() or not frame_queue.empty():
        try:
            frame_id, frame = frame_queue.get(timeout=1)
        except queue.Empty:
            continue

        processed_frames += 1 # Update global counter
        processed_frames_local_for_fps_reporting += 1 # Update local counter for internal reporting

        results = model(frame, verbose=False, device=0 if GPU else "cpu")

        violations = []
        for box in results[0].boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            label = names[cls_id]

            if "no" in label.lower() or "without" in label.lower():
                timestamp = datetime.now(timezone.utc).isoformat()
                violations.append({
                    "frame": frame_id,
                    "label": label,
                    "confidence": round(conf, 3),
                    "timestamp": timestamp
                })
                alert_mgr.handle_violation({
                    "video": os.path.basename(VIDEO_PATH),
                    "frame": frame_id,
                    "violations": [label],
                    "timestamp": timestamp,
                    "confidence": round(conf, 3)
                })

        # Update FPS every 20 frames (using local counters for real-time reporting)
        if processed_frames_local_for_fps_reporting % 20 == 0:
            elapsed = time.time() - start_time_local_for_fps_reporting
            fps = processed_frames_local_for_fps_reporting / elapsed
            print(f"[INFO] FPS: {fps:.2f}")

    print("Detection thread finished.")

reader_thread = threading.Thread(target=video_reader)
detector_thread = threading.Thread(target=detection_worker)

print("Starting real-time PPE monitoring...")
reader_thread.start()
detector_thread.start()

reader_thread.join()
detector_thread.join()
stop_flag.set()

print("All done — optimized real-time detection pipeline complete!")

# Calculate and print final metrics using the global variables
total_time = time.time() - start_time
avg_fps = processed_frames / total_time
print(f"Average FPS: {avg_fps:.2f}")
print(f"GPU: {'Enabled' if GPU else 'Disabled'}")

AlertManager initialized.
Starting real-time PPE monitoring...
Alert triggered for violation: {'video': '1100493329-preview.mp4', 'frame': 3, 'violations': ['NO-Safety Vest'], 'timestamp': '2025-11-07T13:08:04.179835+00:00', 'confidence': 0.708}
Alert triggered for violation: {'video': '1100493329-preview.mp4', 'frame': 3, 'violations': ['NO-Safety Vest'], 'timestamp': '2025-11-07T13:08:04.180411+00:00', 'confidence': 0.513}
Alert triggered for violation: {'video': '1100493329-preview.mp4', 'frame': 6, 'violations': ['NO-Safety Vest'], 'timestamp': '2025-11-07T13:08:04.484018+00:00', 'confidence': 0.62}
Alert triggered for violation: {'video': '1100493329-preview.mp4', 'frame': 6, 'violations': ['NO-Safety Vest'], 'timestamp': '2025-11-07T13:08:04.484500+00:00', 'confidence': 0.61}
Alert triggered for violation: {'video': '1100493329-preview.mp4', 'frame': 9, 'violations': ['NO-Safety Vest'], 'timestamp': '2025-11-07T13:08:04.790909+00:00', 'confidence': 0.846}
Alert triggered for viol